In [2]:

sqlContext.sql("use retail_db_txt").show()
sqlContext.sql("show tables").show()
sqlContext.sql("select distinct order_status from orders limit 10")
sqlContext.sql("select order_status , case order_status when 'COMPLETE' then 'No Action' \
                                         when 'CLOSED' then 'No Action' \
                                          else 'Risky' end \
                                         from orders")

sqlContext.sql("select order_status , case when order_status IN ('COMPLETE', 'CLOSED') then 'No Action' \
                                           when order_status IN ('PENDING_PAYMENT') then 'Pending' \
                                           else 'Risky' end \
                                         from orders")
sqlContext.sql("select nvl(order_status,'Status Missing') from orders limit 100")

sqlContext.sql("select cast(concat(substr(order_date,1,4),substr(order_date,6,2)) as int) from orders")

sqlContext.sql("select date_format(order_date,'YYYYMM') from orders")

sqlContext.sql("select o.*,oi.order_item_product_id from orders o inner join order_items oi on o.order_id = order_item_order_id")

sqlContext.sql("select order_status,count(order_status) from orders group by order_status"))

d = sqlContext.sql("select order_id,order_status, sum(order_item_subtotal) from orders o inner join order_items oi on o.order_id = oi.order_item_order_id group by order_id,order_status")

for i in d.collect():
    if(i.order_id == 42):
        print i
sqlContext.sql("select order_id,order_date, sum(order_item_subtotal) as revenue from orders o inner join order_items oi on o.order_id = oi.order_item_order_id group by order_id,order_date having sum(order_item_subtotal) >= 1000 order by revenue,order_date desc")

sqlContext.sql("select order_id,order_date, sum(order_item_subtotal) as revenue from orders o inner join order_items oi on o.order_id = oi.order_item_order_id group by order_id,order_date having sum(order_item_subtotal) >= 1000 distribute by o.order_date sort by order_date,revenue desc")

#union all for all records
sqlContext.sql("select 1, 'Hello' union all select 2, 'world' union all select 3, 'Hello'")

#union for disctinct
#sqlContext.sql("select 1, 'Hello' union select 2, 'world' union select 3, 'Hello'")

l = sqlContext.sql("select order_id,order_date,order_item_subtotal, round(sum(order_item_subtotal) over (partition by order_id),2) as revenue, order_item_subtotal/round(sum(order_item_subtotal) over (partition by order_id),2) as percentage_revenue from orders o inner join order_items oi on o.order_id = oi.order_item_order_id order by revenue,order_date desc")
for i in l.take(50):print i

++
||
++
++

+-----------+-----------+
|  tableName|isTemporary|
+-----------+-----------+
|  customers|      false|
|order_items|      false|
|     orders|      false|
+-----------+-----------+

+---------------+-------------------------------------------------------------------------------------------------------------+
|   order_status|CASE WHEN (order_status = COMPLETE) THEN No Action WHEN (order_status = CLOSED) THEN No Action ELSE Risky END|
+---------------+-------------------------------------------------------------------------------------------------------------+
|         CLOSED|                                                                                                    No Action|
|PENDING_PAYMENT|                                                                                                        Risky|
|       COMPLETE|                                                                                                    No Action|
|         CLOSED|                   

In [ ]:
l = sqlContext.sql("select 1, 'Hello' union all select 2, 'world' union all select 3, 'Hello'").show()
print "sdas"

In [1]:
sqlContext.sql("use retail_db_txt")
sqlContext.sql("show tables").show()

+-----------+-----------+
|  tableName|isTemporary|
+-----------+-----------+
|  customers|      false|
|order_items|      false|
|     orders|      false|
+-----------+-----------+



In [39]:
l = sqlContext.sql("select order_id,order_date,order_item_subtotal, round(sum(order_item_subtotal) over (partition by order_id),2) as revenue, order_item_subtotal/round(sum(order_item_subtotal) over (partition by order_id),2) as percentage_revenue from orders o inner join order_items oi on o.order_id = oi.order_item_order_id \
order by revenue,order_date desc")
for i in l.take(2):
    if i.order_id == 8210:
        print i

### rank() , dense_rank , percentage_rank, row_number


In [40]:
l = sqlContext.sql("select * from (\
                     select order_id,order_date,order_item_subtotal, \
                     round(sum(order_item_subtotal) over (partition by order_id),2) as order_revenue, \
                     order_item_subtotal/round(sum(order_item_subtotal) over (partition by order_id),2) \
                     as percentage_revenue, \
                     rank() over(partition by oi.order_item_order_id order by oi.order_item_subtotal desc) r_revenue, \
                     dense_rank() over(partition by oi.order_item_order_id order by oi.order_item_subtotal desc) d_revenue, \
                     percent_rank() over(partition by oi.order_item_order_id order by oi.order_item_subtotal desc) p_revenue, \
                     row_number() over(partition by oi.order_item_order_id order by oi.order_item_subtotal desc) rn_revenue \
                     from orders o inner join order_items oi on o.order_id = oi.order_item_order_id \
                     where o.order_status in ('COMPLETE', 'CLOSED')) q where q.order_revenue >= 1000 \
                     order by q.order_revenue,q.order_date desc, q.rn_revenue ")
for i in l.take(50):
    if i.order_id == 8210:
        print i

Row(order_id=8210, order_date=u'2013-09-14 00:00:00.0', order_item_subtotal=399.9800109863281, order_revenue=1004.92, percentage_revenue=0.3980217440058195, r_revenue=1, d_revenue=1, p_revenue=0.0, rn_revenue=1)
Row(order_id=8210, order_date=u'2013-09-14 00:00:00.0', order_item_subtotal=399.9800109863281, order_revenue=1004.92, percentage_revenue=0.3980217440058195, r_revenue=1, d_revenue=1, p_revenue=0.0, rn_revenue=2)
Row(order_id=8210, order_date=u'2013-09-14 00:00:00.0', order_item_subtotal=129.99000549316406, order_revenue=1004.92, percentage_revenue=0.12935358585077825, r_revenue=3, d_revenue=2, p_revenue=0.6666666666666666, rn_revenue=3)
Row(order_id=8210, order_date=u'2013-09-14 00:00:00.0', order_item_subtotal=74.97000122070312, order_revenue=1004.92, percentage_revenue=0.07460295468365952, r_revenue=4, d_revenue=3, p_revenue=1.0, rn_revenue=4)


### lead lag, first, last

In [42]:
l = sqlContext.sql("select * from (\
                     select order_id,order_date,order_item_subtotal, \
                     round(sum(order_item_subtotal) over (partition by order_id),2) as order_revenue, \
                     order_item_subtotal/round(sum(order_item_subtotal) over (partition by order_id),2) \
                     as percentage_revenue, \
                     rank() over(partition by oi.order_item_order_id order by oi.order_item_subtotal desc) r_revenue, \
                     dense_rank() over(partition by oi.order_item_order_id order by oi.order_item_subtotal desc) d_revenue, \
                     percent_rank() over(partition by oi.order_item_order_id order by oi.order_item_subtotal desc) p_revenue, \
                     row_number() over(partition by oi.order_item_order_id order by oi.order_item_subtotal desc) rn_revenue, \
                     lead(oi.order_item_subtotal) over (partition by o.order_id order by oi.order_item_subtotal desc) lead_subtotlast_subtotal, \
                     lag(oi.order_item_subtotal) over (partition by o.order_id order by oi.order_item_subtotal desc) lag_subtotal, \
                     first_value(oi.order_item_subtotal) over (partition by o.order_id order by oi.order_item_subtotal desc) first_subtotal, \
                     last_value(oi.order_item_subtotal) over (partition by o.order_id order by oi.order_item_subtotal desc) last_subtotal \
                     from orders o inner join order_items oi on o.order_id = oi.order_item_order_id \
                     where o.order_status in ('COMPLETE', 'CLOSED')) q where q.order_revenue >= 1000 \
                     order by q.order_revenue,q.order_date desc, q.rn_revenue ")
for i in l.take(50):
    if i.order_id == 8210:
        print i

Row(order_id=8210, order_date=u'2013-09-14 00:00:00.0', order_item_subtotal=399.9800109863281, order_revenue=1004.92, percentage_revenue=0.3980217440058195, r_revenue=1, d_revenue=1, p_revenue=0.0, rn_revenue=1, lead_subtotlast_subtotal=399.9800109863281, lag_subtotal=None, first_subtotal=399.9800109863281, last_subtotal=399.9800109863281)
Row(order_id=8210, order_date=u'2013-09-14 00:00:00.0', order_item_subtotal=399.9800109863281, order_revenue=1004.92, percentage_revenue=0.3980217440058195, r_revenue=1, d_revenue=1, p_revenue=0.0, rn_revenue=2, lead_subtotlast_subtotal=129.99000549316406, lag_subtotal=399.9800109863281, first_subtotal=399.9800109863281, last_subtotal=399.9800109863281)
Row(order_id=8210, order_date=u'2013-09-14 00:00:00.0', order_item_subtotal=129.99000549316406, order_revenue=1004.92, percentage_revenue=0.12935358585077825, r_revenue=3, d_revenue=2, p_revenue=0.6666666666666666, rn_revenue=3, lead_subtotlast_subtotal=74.97000122070312, lag_subtotal=399.980010986328

In [55]:
#sqlContext.sql("create database retail_db_orc")
sqlContext.sql("use retail_db_orc")
#sqlContext.sql("create table orders (order_id int, order_date string, order_customer_id int,order_status string ) stored as orc")
#sqlContext.sql("load data local inpath '/home/zubairidrees/hadoop-2.7.3/data/retail_db/orders' into table orders")
#sqlContext.sql("create table order_items (order_item_id int, order_item_order_id int, order_item_product_id int, order_item_quantity int,order_item_subtotal float, order_item_product_price float) stored as orc")
sqlContext.sql("insert into retail_db_orc.orders from select * from retail_db_txt.orders")
sqlContext.sql("insert into retail_db_orc.order_items from select * from retail_db_txt.order_items")


#sqlContext.sql("load data local inpath '/home/zubairidrees/hadoop-2.7.3/data/retail_db/orders' into table orders")

ParseException: u"\nextraneous input '*' expecting {<EOF>, ',', 'SELECT', 'FROM', 'ADD', 'AS', 'ALL', 'DISTINCT', 'WHERE', 'GROUP', 'BY', 'GROUPING', 'SETS', 'CUBE', 'ROLLUP', 'ORDER', 'HAVING', 'LIMIT', 'AT', 'OR', 'AND', 'IN', NOT, 'NO', 'EXISTS', 'BETWEEN', 'LIKE', RLIKE, 'IS', 'NULL', 'TRUE', 'FALSE', 'NULLS', 'ASC', 'DESC', 'FOR', 'INTERVAL', 'CASE', 'WHEN', 'THEN', 'ELSE', 'END', 'JOIN', 'CROSS', 'OUTER', 'INNER', 'LEFT', 'RIGHT', 'FULL', 'NATURAL', 'LATERAL', 'WINDOW', 'OVER', 'PARTITION', 'RANGE', 'ROWS', 'UNBOUNDED', 'PRECEDING', 'FOLLOWING', 'CURRENT', 'ROW', 'WITH', 'VALUES', 'CREATE', 'TABLE', 'VIEW', 'REPLACE', 'INSERT', 'DELETE', 'INTO', 'DESCRIBE', 'EXPLAIN', 'FORMAT', 'LOGICAL', 'CODEGEN', 'CAST', 'SHOW', 'TABLES', 'COLUMNS', 'COLUMN', 'USE', 'PARTITIONS', 'FUNCTIONS', 'DROP', 'UNION', 'EXCEPT', 'INTERSECT', 'TO', 'TABLESAMPLE', 'STRATIFY', 'ALTER', 'RENAME', 'ARRAY', 'MAP', 'STRUCT', 'COMMENT', 'SET', 'RESET', 'DATA', 'START', 'TRANSACTION', 'COMMIT', 'ROLLBACK', 'MACR

In [57]:
sqlContext.sql("use retail_db_orc")
sqlContext.sql("show tables").show()
sqlContext.sql("select * from orders limit 10").printSchema()

+-----------+-----------+
|  tableName|isTemporary|
+-----------+-----------+
|order_items|      false|
|     orders|      false|
+-----------+-----------+

root
 |-- order_id: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_customer_id: integer (nullable = true)
 |-- order_status: string (nullable = true)




### Read data from file, create dataframe & create temp table 

In [4]:
#from pyspark.sql import Row
orders =sc.textFile("/retail_db/orders")
for i in orders.take(5): print i

Py4JJavaError: An error occurred while calling o64.partitions.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: hdfs://localhost:9000/retail_db/orders
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:287)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:199)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:248)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:246)
	at scala.Option.getOrElse(Option.scala:121)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:246)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:35)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:248)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:246)
	at scala.Option.getO

### Create DataFrame & temp table

In [22]:
from pyspark.sql import Row
ordersRDD = sc.textFile("retail_db/orders")
ordersDF = ordersRDD.map(lambda o: Row(id=int(o.split(",")[0] ), j=o.split(",")[1], price=o.split(",")[2], status=o.split(",")[3] ) ).toDF()
#ordersDF.show()
ordersDF.registerTempTable("orders_tmp")
ordersCountByStatus = sqlContext.sql("select count(status),status from orders_tmp group by status")
for i in ordersCountByStatus.take(30):
    print i
    
productsRaw = open("/home/zubairidrees/hadoop-2.7.3/data/retail_db/products/part-00000").read().splitlines()
type(productsRaw)
productsRDD = sc.parallelize(productsRaw)
for i in productsRDD.take(5):print i

Row(count(status)=15030, status=u'PENDING_PAYMENT')
Row(count(status)=22899, status=u'COMPLETE')
Row(count(status)=3798, status=u'ON_HOLD')
Row(count(status)=729, status=u'PAYMENT_REVIEW')
Row(count(status)=8275, status=u'PROCESSING')
Row(count(status)=7556, status=u'CLOSED')
Row(count(status)=1558, status=u'SUSPECTED_FRAUD')
Row(count(status)=7610, status=u'PENDING')
Row(count(status)=1428, status=u'CANCELED')
1,2,Quest Q64 10 FT. x 10 FT. Slant Leg Instant U,,59.98,http://images.acmesports.sports/Quest+Q64+10+FT.+x+10+FT.+Slant+Leg+Instant+Up+Canopy
2,2,Under Armour Men's Highlight MC Football Clea,,129.99,http://images.acmesports.sports/Under+Armour+Men%27s+Highlight+MC+Football+Cleat
3,2,Under Armour Men's Renegade D Mid Football Cl,,89.99,http://images.acmesports.sports/Under+Armour+Men%27s+Renegade+D+Mid+Football+Cleat
4,2,Under Armour Men's Renegade D Mid Football Cl,,89.99,http://images.acmesports.sports/Under+Armour+Men%27s+Renegade+D+Mid+Football+Cleat
5,2,Riddell Youth Revol

In [46]:
from pyspark.sql import Row
orders = sc.textFile("retail_db/orders")
order_items = sc.textFile("retail_db/order_items")
products = sc.textFile("retail_db/products")

ordersDF = orders.map(lambda o: Row(order_id=int(o.split(",")[0]) , order_date=o.split(",")[1], \
                                    order_status=o.split(",")[3])).toDF()

orderItemsDF = order_items.map(lambda oi:Row(order_item_id=int(oi.split(",")[0]) , order_id=int(oi.split(",")[1]), \
                                             product_id=int(oi.split(",")[2]), revenue=float(oi.split(",")[4]) ) ).toDF()
productsDF = products.map(lambda p:Row(product_id=int(p.split(",")[0]) , product_name=p.split(",")[2])).toDF()

ordersDF.show()

print orders.take(1)
print order_items.take(1)
print products.take(1)

ordersDF.registerTempTable("orders")
orderItemsDF.registerTempTable("order_items")
productsDF.registerTempTable("products")

sqlContext.setConf("spark.sql.shuffle.partitions",2)
sqlContext.sql("select o.order_date, p.product_name ,sum(oi.revenue) as daily_revenue \
                from orders o join order_items oi on o.order_id = oi.order_id \
                join products p on p.product_id = oi.product_id \
                where o.order_status in ('COMPLETE','CLOSED') \
                group by o.order_date, p.product_name \
                order by o.order_date ,daily_revenue  desc").show()





+--------------------+--------+---------------+
|          order_date|order_id|   order_status|
+--------------------+--------+---------------+
|2013-07-25 00:00:...|       1|         CLOSED|
|2013-07-25 00:00:...|       2|PENDING_PAYMENT|
|2013-07-25 00:00:...|       3|       COMPLETE|
|2013-07-25 00:00:...|       4|         CLOSED|
|2013-07-25 00:00:...|       5|       COMPLETE|
|2013-07-25 00:00:...|       6|       COMPLETE|
|2013-07-25 00:00:...|       7|       COMPLETE|
|2013-07-25 00:00:...|       8|     PROCESSING|
|2013-07-25 00:00:...|       9|PENDING_PAYMENT|
|2013-07-25 00:00:...|      10|PENDING_PAYMENT|
|2013-07-25 00:00:...|      11| PAYMENT_REVIEW|
|2013-07-25 00:00:...|      12|         CLOSED|
|2013-07-25 00:00:...|      13|PENDING_PAYMENT|
|2013-07-25 00:00:...|      14|     PROCESSING|
|2013-07-25 00:00:...|      15|       COMPLETE|
|2013-07-25 00:00:...|      16|PENDING_PAYMENT|
|2013-07-25 00:00:...|      17|       COMPLETE|
|2013-07-25 00:00:...|      18|         